In [2]:
from xarm.wrapper import XArmAPI
from time import sleep
import numpy as np
import cv2
from random import randint

arm = XArmAPI('192.168.1.155')
arm.motion_enable(enable=True)
arm.set_mode(0)
arm.set_state(0)
arm.connect()
arm.move_gohome()

SDK_VERSION: 1.13.30
ROBOT_IP: 192.168.1.155, VERSION: v2.2.0, PROTOCOL: V1, DETAIL: 6,9,LI1006,DL1000,v2.2.0, TYPE1300: [0, 0]
change protocol identifier to 3


0

In [3]:
def mouse_callback(event: int, x: int, y: int, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        global point
        point = (y, x)
        # print(f"Point clicked: ({x}, {y})")


def pickup_block():
    # x, y, z = 225, -95, 87
    arm.move_gohome()
    arm.set_suction_cup(True)
    sleep(4)


def pickup_block_from(x, y, z=20) -> bool:
    """
    Returns false if out of bounds
    :param x: 
    :param y: 
    :param z: 
    :return: 
    """
    if not (215 < x < 300) or y < 0:
        print("Out of RANGE")
        return False
    arm.set_suction_cup(True)
    arm.set_position(x=x, y=y, z=z + 10, roll=180, pitch=0, yaw=0)
    sleep(0.5)
    arm.set_position(x=x, y=y, z=z, roll=180, pitch=0, yaw=0)
    sleep(1)
    arm.set_position(x=x, y=y, z=z + 40, roll=180, pitch=0, yaw=0)
    arm.move_gohome()
    return True


def place_block(x, y, z=21):
    arm.set_position(x=x, y=y, z=z, roll=180, pitch=0, yaw=0)
    arm.set_suction_cup(False)
    arm.set_position(x=x, y=y, z=z + 10, roll=180, pitch=0, yaw=0)
    arm.move_gohome()


def place_block_on(x=225, y=-95, z=87):
    arm.move_gohome()
    arm.set_position(x, y, z, roll=180, pitch=0, yaw=0)
    arm.set_suction_cup(False)
    arm.move_gohome()


def add_rows(matrix, row):
    if matrix.size == 0:
        return np.array(row).reshape(1, -1)
    else:
        return np.vstack((matrix, np.array(row)))

In [4]:
Rx = np.array([])
Ry = np.array([])
Px = np.array([])
Py = np.array([])

In [5]:
cap = cv2.VideoCapture(1)  # Use 0 for the default camera, or provide a video file path
cap.set(3, 640)
cap.set(4, 480)
cv2.namedWindow("Video")
cv2.setMouseCallback("Video", mouse_callback)

In [8]:
for i in range(25):
    pickup_block()
    bot_x, bot_y = (randint(215, 275), randint(0, 300))
    place_block(bot_x, bot_y, z=27)

    point: tuple[int, int] | None = None

    while not point:
        ret, frame = cap.read()
        cv2.imshow("Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    if not point:
        break

    print(f"Robot: {(bot_x, bot_y)}, Pixel: {point}")
    with open("points.txt", "a") as f:
        f.write(f"{(bot_x, bot_y)},{point}\n")

    # Rx = add_rows(Rx, [bot_x])
    # Ry = add_rows(Ry, [bot_y])
    # Px = add_rows(Px, [point[0], 1])
    # Py = add_rows(Py, [point[1], 1])

Robot: (259, 221), Pixel: (160, 471)
Robot: (245, 235), Pixel: (137, 490)
Robot: (266, 277), Pixel: (168, 553)
Robot: (245, 159), Pixel: (142, 375)
Robot: (273, 139), Pixel: (187, 345)
Robot: (227, 251), Pixel: (111, 513)
Robot: (265, 203), Pixel: (172, 444)
Robot: (266, 148), Pixel: (177, 359)
Robot: (232, 110), Pixel: (127, 300)
Robot: (267, 111), Pixel: (178, 305)
Robot: (252, 154), Pixel: (155, 368)
Robot: (246, 204), Pixel: (144, 443)
Robot: (261, 38), Pixel: (174, 192)
Robot: (244, 5), Pixel: (154, 144)
Robot: (252, 111), Pixel: (158, 301)
Robot: (269, 164), Pixel: (181, 382)
Robot: (236, 14), Pixel: (138, 156)
Robot: (258, 219), Pixel: (160, 468)
Robot: (215, 50), Pixel: (106, 210)
Robot: (264, 47), Pixel: (182, 204)
Robot: (232, 249), Pixel: (121, 511)
Robot: (229, 164), Pixel: (119, 380)
Robot: (217, 138), Pixel: (103, 338)
Robot: (223, 30), Pixel: (119, 180)


In [9]:
with open("points.txt") as f:
    for line in set(f.readlines()):
        bot_coords, pixel_coords = eval(line)
        Rx = add_rows(Rx, [bot_coords[0]])
        Ry = add_rows(Ry, [bot_coords[1]])
        Px = add_rows(Px, [pixel_coords[0], 1])
        Py = add_rows(Py, [pixel_coords[1], 1])


In [10]:
print(Rx.size)
print(Rx, Px)
print(Ry, Py)
m1, c1 = np.linalg.pinv(Px) @ Rx
m2, c2 = np.linalg.pinv(Py) @ Ry

m1, c1 = round(m1[0], 5), round(c1[0], 5)
m2, c2 = round(m2[0], 5), round(c2[0], 5)
print(f"m1,c1 = {m1},{c1}")
print(f"m2,c2 = {m2},{c2}")

75
[[260]
 [264]
 [245]
 [247]
 [270]
 [237]
 [266]
 [236]
 [257]
 [223]
 [262]
 [273]
 [227]
 [244]
 [216]
 [266]
 [216]
 [262]
 [263]
 [269]
 [229]
 [245]
 [220]
 [265]
 [230]
 [240]
 [246]
 [230]
 [261]
 [259]
 [223]
 [265]
 [228]
 [246]
 [270]
 [256]
 [267]
 [242]
 [252]
 [261]
 [268]
 [238]
 [245]
 [217]
 [244]
 [250]
 [256]
 [227]
 [269]
 [255]
 [236]
 [236]
 [267]
 [232]
 [265]
 [245]
 [232]
 [259]
 [242]
 [215]
 [261]
 [239]
 [258]
 [229]
 [263]
 [252]
 [216]
 [249]
 [271]
 [230]
 [234]
 [242]
 [265]
 [228]
 [260]] [[159   1]
 [182   1]
 [137   1]
 [154   1]
 [175   1]
 [140   1]
 [177   1]
 [123   1]
 [162   1]
 [119   1]
 [167   1]
 [187   1]
 [111   1]
 [154   1]
 [101   1]
 [168   1]
 [111   1]
 [178   1]
 [167   1]
 [181   1]
 [126   1]
 [139   1]
 [116   1]
 [186   1]
 [118   1]
 [140   1]
 [154   1]
 [121   1]
 [173   1]
 [160   1]
 [110   1]
 [181   1]
 [118   1]
 [144   1]
 [179   1]
 [154   1]
 [174   1]
 [139   1]
 [158   1]
 [174   1]
 [172   1]
 [126   1]
 [142   1

In [11]:
# Hand Calculated (2 blocks):
# m1, c1 = 0.7251, 179.5495
# m2, c2 = 0.6383, -82.3553
# 10 blocks:
# m1, c1 = 0.71108,179.96984
# m2,c2 = 0.63734, -87.31099
# 39 blocks:
# m1,c1 = 0.56171, 197.02872
# m2,c2 = 0.64017,-85.75084
# 50 blocks:
# m1, c1 = 0.54665, 198.17924
# m2, c2 = 0.64072, -86.07211
# Reposition - 50 blocks
# m1, c1 = 0.6519, 150.83156
# m2,c2 = 0.64737, -84.42875
# Reposition - 50 blocks
# m1, c1 = 0.6456, 151.74737
# m2, c2 = 0.65968, -88.87191
# Same - 75 blocks
m1,c1 = 0.6515,150.83251
m2,c2 = 0.65973,-88.76217
pixel_to_world = lambda px, py: (m1 * px + c1, m2 * py + c2)
# pixel_to_world(133,136)

In [12]:
while True:
    point = None
    while point is None:
        ret, frame = cap.read()
        # Display the frame
        cv2.imshow("Video", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        rx, ry = pixel_to_world(point[0], point[1])
        print("Pixel: ", point)
        print(f"World: ({rx}, {ry})")
        if pickup_block_from(rx, ry):
            # sleep(2)
            place_block_on()

Pixel:  (135, 140)
World: (238.78501, 3.600030000000004)
Pixel:  (117, 196)
World: (227.05801000000002, 40.544910000000016)
Pixel:  (173, 189)
World: (263.54201, 35.926800000000014)
Pixel:  (174, 249)
World: (264.19351, 75.51060000000001)
Pixel:  (116, 260)
World: (226.40651000000003, 82.76763000000003)
Pixel:  (132, 310)
World: (236.83051, 115.75413)
Pixel:  (105, 358)
World: (219.24001, 147.42117000000002)
Pixel:  (114, 409)
World: (225.10351000000003, 181.0674)
Pixel:  (172, 392)
World: (262.89051, 169.85199000000003)
Pixel:  (179, 456)
World: (267.45101, 212.07471)
Pixel:  (121, 464)
World: (229.66401000000002, 217.35255000000004)
Pixel:  (128, 502)
World: (234.22451, 242.42229)
Pixel:  (96, 573)
World: (213.37651, 289.2631200000001)
Out of RANGE
Pixel:  (110, 626)
World: (222.49751, 324.22881000000007)
Pixel:  (167, 585)
World: (259.63301, 297.17988)
Pixel:  (143, 544)
World: (243.99701, 270.13095)
Pixel:  (187, 220)
World: (272.66301, 56.37843000000001)
Pixel:  (105, 274)
World: 

KeyboardInterrupt: 

In [2]:
arm.set_suction_cup(False)
sleep(2)
arm.move_gohome()

0

In [24]:
pickup_block()
place_block(430, -16)